In [1]:
WORKING_DIR = '/Users/nikunjkumarraval/OneDrive/projects/twitter'

TWITTER_API_SECRET_1="DVYV0jX3GI3ZGzbU9jqBjnfx6"
TWITTER_API_SECRET_2="4byshg5kGXWTZdsDKJLwKGvSDIuLKmcuhE5Q2OgG4Pffdu3Tpw"

INTERESTED_PEOPLE = ["RayDalio", "larryellison", "GinniRometty", "ericschmidt", "marceloclaure", "sarandos_ted", "Benioff"]
INTERESTED_COMPAINES = ["Coinbase", "OpenAI"]
INTERESTED_PROFILES = INTERESTED_PEOPLE + INTERESTED_COMPAINES

IMG_FILE_IDX=0

In [2]:
import tweepy
import json
import pprint
import pandas as pd
import urllib
import os
import glob
import random
import base64
import pandas as pd
import time
import requests
import json

from IPython.display import HTML

pd.set_option('display.max_colwidth', -1)
 
TPAPI = tweepy.API(tweepy.OAuthHandler(TWITTER_API_SECRET_1, TWITTER_API_SECRET_2))
IMG_FILE_IDX = 1

/tmp/ipykernel_3198/3250034511.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
## Function to download basic info
def get_basic_info():
    basic_info = {}
    for profile in INTERESTED_PROFILES:
        time.sleep(1)
        response_json = TPAPI.get_user(screen_name=profile)._json
        interested_fields = {
            'id': response_json['id'],
            'name': response_json['name'],
            'description': response_json['description'],
            'image': response_json['profile_image_url'],
            'followers_count': response_json['followers_count'],
            'friends_count': response_json['friends_count'],
        }
        basic_info[profile] = interested_fields
    df = pd.DataFrame(basic_info).T
    return df

df_basic_info = get_basic_info()
HTML(df_basic_info[["name", "description", "image", "followers_count", "friends_count"]].to_html(
    escape=False, formatters=dict(image=lambda p : '<img src="'+ p + '" width="60" >')))

,name,description,image,followers_count,friends_count
RayDalio,Ray Dalio,"Official account of Ray Dalio, founder of Bridgewater Associates, author of #1 New York Times bestseller 'Principles,' professional mistake maker",,776665,92
larryellison,Larry Ellison,,,113291,41
GinniRometty,Ginni Rometty,Living my values. Former chairman and CEO of @IBM. I believe in the power of business and technology to create a better tomorrow.,,35739,93
ericschmidt,Eric Schmidt,Former Executive Chairman & CEO and tweets from Schmidt Foundation,,2231173,143
marceloclaure,Marcelo Claure,"Father of 6,Husband,CEO @Softbank Int’l, Chairman @WeWork & Fortress. Board @Tmobile,@Arm,@Univision, @SorareHQ, @Bolivar_Oficial & @gironafc ⚽️",,281322,868
sarandos_ted,Ted Sarandos,,,635,0
Benioff,Marc Benioff,ceo@salesforce.com chair@time.com,,1062826,3725
Coinbase,Coinbase,Coinbase is the easiest place to buy and sell crypto. For support ➡️ @CoinbaseSupport,,4549378,21
OpenAI,OpenAI,OpenAI’s mission is to ensure that artificial general intelligence benefits all of humanity. We’re hiring: https://t.co/XxM7UoAwz9,,639319,0


In [8]:
def get_friends():
    result = {}
    for profile in INTERESTED_PROFILES:
        time.sleep(1)
        response_json = TPAPI.get_friends(screen_name=profile)
        friends = []
        for friend in response_json:
            interested_fields = {
                'id': friend._json['id'],
                'name': friend._json['name'],
                'description': friend._json['description'],
                'image': friend._json['profile_image_url'],
                'followers_count': friend._json['followers_count'],
                'friends_count': friend._json['friends_count'],
            }
            friends.append(interested_fields)
        result[profile] = pd.DataFrame(friends)
    return result

friends = get_friends()

In [7]:
def get_tweets_per_user(profile):
    tweets = []
    for tweet in tweepy.Cursor(TPAPI.user_timeline,screen_name=profile, tweet_mode = 'extended').items():
        t = {
            'created_at': tweet._json['full_text'],
            'full_text': tweet._json['full_text'],
            'retweet_count': tweet._json['retweet_count'],
            'favorite_count': tweet._json['favorite_count']
        }
        tweets.append(tweet)
    return tweets

def get_tweets():
    tweets = {}
    for profile in INTERESTED_PROFILES:
        print(f'Getting tweeets for {profile}')
        tweets[profile] = get_tweets_per_user(profile)
        print(f'Got {len(tweets[profile])} tweeets for {profile}')
    return tweets

tweets = get_tweets()

Getting tweeets for RayDalio
Got 3250 tweeets for RayDalio
Getting tweeets for larryellison
Got 1 tweeets for larryellison
Getting tweeets for GinniRometty
Got 156 tweeets for GinniRometty
Getting tweeets for ericschmidt
Got 979 tweeets for ericschmidt
Getting tweeets for marceloclaure
Got 3249 tweeets for marceloclaure
Getting tweeets for sarandos_ted
Got 0 tweeets for sarandos_ted
Getting tweeets for Benioff
Got 3244 tweeets for Benioff
Getting tweeets for Coinbase
Got 1499 tweeets for Coinbase
Getting tweeets for OpenAI
Got 526 tweeets for OpenAI


In [9]:
df_basic_info.to_csv('./basic_info.csv', header=True, index=True)

In [10]:
os.makedirs('./tweets', exist_ok=True)
for profile in tweets.keys():
    pd.DataFrame(tweets[profile]).to_csv(f'./tweets/{profile}.csv', index=False, header=False)

In [11]:
os.makedirs('./friends', exist_ok=True)
for profile in friends.keys():
    friends[profile].to_csv(f'./friends/{profile}.csv', index=False, header=True)

In [12]:
result = []
for p in INTERESTED_PROFILES:
    fs = friends[p].T.to_dict() 
    result.append({
        'id': p,
        'name': df_basic_info.T.to_dict()[p]['name'],
        'basic_info': df_basic_info.T.to_dict()[p],
        'friends': [fs[key] for key in fs.keys()],
        'tweets': tweets[p]
    })
with open('./result.json', 'w') as fp:
    json.dump(result, fp, indent=4 * ' ')

TypeError: Object of type Status is not JSON serializable